In [19]:
!pip install scikit-learn
!pip install pandas
!pip install nltk

765.56s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


771.51s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


777.36s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [20]:
import pandas as pd
import numpy as np
import ast
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
movies =pd.read_csv("/Users/bhavneetsinghsahni/Desktop/Site/tmdb_5000_movies.csv")

In [22]:
credits=pd.read_csv("/Users/bhavneetsinghsahni/Desktop/Site/tmdb_5000_credits.csv")

In [23]:
movies.shape

(4803, 20)

In [24]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [25]:
movies=movies.merge(credits, on="title")

In [26]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""bame"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [27]:
#remove any column that cannot be used in creating a tag
movies=movies[["movie_id", "title","genres","overview", "keywords", "cast", "crew"]]

In [28]:
movies.head()

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[{""id"": 28, ""bame"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [29]:
movies.isnull().sum()   

movie_id    0
title       0
genres      0
overview    3
keywords    0
cast        0
crew        0
dtype: int64

In [30]:
movies.dropna(inplace=True)

In [31]:
movies.duplicated().sum()

0

In [32]:
def convert(obj):
    list=[]
    for i in ast.literal_eval(obj):
        list.append(i["name"])
    return list

In [33]:
 # when using apply the function is called for each element seprately otherwise it would pass in form of json format

import ast

def convert(obj):
    genre_list = []
    for i in ast.literal_eval(obj):
        if "name" in i:  # Check if the key "name" exists in the dictionary
            genre_list.append(i["name"])  # Access the value associated with the key "name"
    return genre_list


In [34]:
movies["keywords"]=movies["keywords"].apply(convert)

In [35]:
movies.head(1)

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[{""id"": 28, ""bame"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [36]:
def convert_cast(obj):
    list=[]
    count=0
    for i in ast.literal_eval(obj):
        list.append(i["name"])
        count+=1
        if(count==3):
            break;
    return list

In [37]:
movies["cast"]=movies["cast"].apply(convert_cast)

In [38]:
def convert_crew(obj):
    list=[]
    count=0
    for i in ast.literal_eval(obj):
        if(i["job"].lower()=="director"):
            list.append(i["name"])
            break
    return list


In [39]:
movies["crew"]=movies["crew"].apply(convert_crew)

In [40]:
movies["crew"]

0           [James Cameron]
1          [Gore Verbinski]
2              [Sam Mendes]
3       [Christopher Nolan]
4          [Andrew Stanton]
               ...         
4804     [Robert Rodriguez]
4805         [Edward Burns]
4806          [Scott Smith]
4807          [Daniel Hsia]
4808     [Brian Herzlinger]
Name: crew, Length: 4806, dtype: object

In [41]:
movies.head(1)

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[{""id"": 28, ""bame"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [42]:
def remove_space(obj):
    list=[]
    for i in obj:
        list.append(i.replace(" ",""))
    return list

In [43]:
movies["genres"]=movies["genres"].apply(remove_space)

In [44]:
movies["keywords"]=movies["keywords"].apply(remove_space)

In [45]:
movies["cast"]=movies["cast"].apply(remove_space)

In [46]:
movies["crew"]=movies["crew"].apply(remove_space)

In [47]:
movies["overview"]= movies["overview"].apply(lambda x: x.split())

In [48]:
movies.head()

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[[, {, "", i, d, "", :, , 2, 8, ,, , "", b, a, m,...","[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[[, {, "", i, d, "", :, , 1, 2, ,, , "", n, a, m,...","[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[[, {, "", i, d, "", :, , 2, 8, ,, , "", n, a, m,...","[A, cryptic, message, from, Bond’s, past, send...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[[, {, "", i, d, "", :, , 2, 8, ,, , "", n, a, m,...","[Following, the, death, of, District, Attorney...","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[[, {, "", i, d, "", :, , 2, 8, ,, , "", n, a, m,...","[John, Carter, is, a, war-weary,, former, mili...","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [49]:
movies["tags"]=movies["overview"]+movies["genres"]+movies["keywords"]+movies["cast"]+movies["crew"]

In [50]:
movies.head(1)

,movie_id,title,genres,overview,keywords,cast,crew,tags
0,19995,Avatar,"[[, {, "", i, d, "", :, , 2, 8, ,, , "", b, a, m,...","[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [51]:
new_df=movies[["movie_id", "title", "tags"]]

In [52]:
new_df

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [53]:
new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))

/var/folders/r2/5_9xyprs5md1hrjfc87x7xvc0000gn/T/ipykernel_51422/1804341217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))


In [54]:
new_df

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [55]:
new_df["tags"]=new_df["tags"].apply(lambda x: x.lower())

/var/folders/r2/5_9xyprs5md1hrjfc87x7xvc0000gn/T/ipykernel_51422/445237715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(lambda x: x.lower())


In [56]:
new_df

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [57]:
new_df.iloc[0].tags


'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. [ { " i d " :  2 8 ,  " b a m e " :  " a c t i o n " } ,  { " i d " :  1 2 ,  " n a m e " :  " a d v e n t u r e " } ,  { " i d " :  1 4 ,  " n a m e " :  " f a n t a s y " } ,  { " i d " :  8 7 8 ,  " n a m e " :  " s c i e n c e  f i c t i o n " } ] cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [58]:
ps=PorterStemmer()

In [59]:
def stem(obj):
    l=[]
    for i in obj.split():
        l.append(ps.stem(i))
        
    return " ".join(l)

In [60]:
new_df["tags"]= new_df["tags"].apply(stem)

/var/folders/r2/5_9xyprs5md1hrjfc87x7xvc0000gn/T/ipykernel_51422/447794818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]= new_df["tags"].apply(stem)


In [61]:
new_df.iloc[0].tags 

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. [ { " i d " : 2 8 , " b a m e " : " a c t i o n " } , { " i d " : 1 2 , " n a m e " : " a d v e n t u r e " } , { " i d " : 1 4 , " n a m e " : " f a n t a s y " } , { " i d " : 8 7 8 , " n a m e " : " s c i e n c e f i c t i o n " } ] cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [62]:
cv =CountVectorizer(max_features=5000, stop_words="english" )

In [63]:
vectors=cv.fit_transform(new_df["tags"]).toarray()

In [64]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [65]:
features = cv.get_feature_names_out()

In [66]:
similarity=cosine_similarity(vectors)

In [67]:
def recommend(movie):
    # Check if the movie exists in the dataset
    if movie not in new_df["title"].values:
        print(f"Movie '{movie}' not found in the dataset.")
        return
    
    # Get the index of the movie in the DataFrame
    movie_index = new_df[new_df["title"] == movie].index[0]
    
    # Access the similarity scores for the movie
    distances = similarity[movie_index]
    
    # Find the indices of the most similar movies
    movie_list = sorted(enumerate(distances), reverse=True, key=lambda x: x[1])[1:6]
    
    # Print the titles of the most similar movies
    for i, similarity_score in movie_list:
        similar_movie_title = new_df.iloc[i]["title"]
        print(f"Similarity score with '{similar_movie_title}': {similarity_score}")


In [68]:

recommend("Ba")
#sorted(list(enumerate((similarity[0]))),reverse=True,key= lambda x: x[1]) #

Movie 'Ba' not found in the dataset.
